In [1]:
# importamos librerias varias
import pandas as pd
from multiprocessing import Pool
from pandas_ods_reader import read_ods
import glob

In [2]:
%%time
# Cargamos desde los pickles
salientes = pd.read_pickle('salientes_01.pkl')
entrantes = pd.read_pickle('entrantes_01.pkl')

CPU times: user 151 ms, sys: 133 ms, total: 284 ms
Wall time: 283 ms


In [3]:
salientes.sample(20)

,llamantes,destino,llamado,fecha-hora,duracion
52685,1136285452,Tráfico incluid,966808726,2023-08-31 10:49:49,37247
117348,1158739788,Conexión Móvil,939488292,2023-09-04 19:00:37,456
36141,1158740317,Conexión Móvil,903719572,2023-08-12 07:44:26,456
13883,1126977230,Conexión Móvil,414471776,2023-08-09 03:03:14,1715
126710,1126654563,Conexión Móvil,336199723,2023-08-28 20:56:57,2369
133346,1138338731,Conexion Movil,166078015,2023-09-03 23:05:11,8182
159478,1135402527,Tráfico incluid,314689474,2023-08-25 21:24:20,2226
46729,1166034459,Tráfico incluid,273845854,2023-08-09 09:34:34,81065
130486,1176081531,Conexión Móvil,271619301,2023-08-22 18:18:17,920
122359,1155242567,WhatsApp Gratis,218106078,2023-09-12 17:43:07,25006


In [10]:
# sacamos un listado de telefonos y la cantidad de cada uno
resumen_llamantes = salientes['llamantes'].value_counts()
resumen_llamantes

1151467918    79226
1145635562    70662
1165409227    48861
1133294894    45722
1121765241    42413
              ...  
2291501888        1
1154181869        1
1176115172        1
1158764385        1
1121804490        1
Name: llamantes, Length: 69414, dtype: int64

In [13]:
# Convertimos lo anterior a un dataframe
# resumen_salientes = salientes.value_counts().rename_axis('telefonos').reset_index(name='llamadas')
# resumen_salientes = salientes.value_counts().rename_axis('telefonos').to_frame('cantidad')
# resumen_salientes

value_counts = salientes['llamantes'].value_counts(dropna=True, sort=True)
# solution here
df_val_counts = pd.DataFrame(value_counts)
df_value_counts_reset = df_val_counts.reset_index()
df_value_counts_reset.columns = ['llamantes', 'cantidad'] # change column names

In [14]:
df_value_counts_reset

,llamantes,cantidad
0,1151467918,79226
1,1145635562,70662
2,1165409227,48861
3,1133294894,45722
4,1121765241,42413
...,...,...
69409,2291501888,1
69410,1154181869,1
69411,1176115172,1
69412,1158764385,1


In [15]:
df_value_counts_reset['cantidad'].nunique()

1772

In [7]:
# Filtrados en la data del hecho +- 20 minutos
fil_sal_hecho = salientes.loc[(salientes['fecha-hora'] >= '2023-09-01 20:25:00')
                     & (salientes['fecha-hora'] < '2023-09-01 21:05:00')]

In [8]:
fil_sal_hecho

,llamantes,destino,llamado,fecha-hora,duracion
134942,1126843731,Conexión Móvil,935100817,2023-09-01 20:25:00,723
134943,1168848160,Tráfico incluid,204026346,2023-09-01 20:25:00,42372
134944,1170725038,WhatsApp Gratis,44880237,2023-09-01 20:25:01,54626
134945,1170725038,TRAFICO DATOS,44880238,2023-09-01 20:25:01,2277
134946,1138110581,WhatsApp Gratis,321547118,2023-09-01 20:25:02,1947
...,...,...,...,...,...
141517,1121708004,Conexión Móvil,313182071,2023-09-01 21:04:58,170
141518,2974424413,Tráfico incluid,307198256,2023-09-01 21:04:58,40944329
141519,1127304803,TRAFICO DATOS,340722385,2023-09-01 21:04:59,1356
141520,1127304803,Tráfico incluid,340722384,2023-09-01 21:04:59,744995


In [9]:
# Contamos la cantidad de numeros de telefonos unicos
salientes['llamantes'].nunique()

69414

In [26]:
#v = df.Parameter.value_counts()
salientes[salientes['llamantes'].isin(resumen_llamantes.index[resumen_llamantes.lt(2)])]

,Nro. Llamante,Destino,Nro. Llamado,fecha y hora,Duracion (Seg),Id. celda,Descripc. Celda,Latitud,Longitud,Dir Celda,fecha y hora
293,1133078828,Conexion Movil,388515017,13/09/2023 0:01:59,630609082,CF01147L200MC3,Crucero Belgrano,"-34,709811","-58,732226",Avda Mosconi 1815,2023-09-13 00:01:59
360,1167175196,Conexion Movil,419412002,13/09/2023 0:02:25,16735380,CF01212L100MC3,Brigada Aérea,"-34,700357","-58,737708",Curapaligue 1069,2023-09-13 00:02:25
463,1139073245,Conexion Movil,4769023,13/09/2023 0:03:33,1280277205,CF01212L100MC2,BRIGADA AÉREA,"-34,700405","-58,737688",Curapaligue 1069,2023-09-13 00:03:33
1284,1171094220,Conexion Movil,1789312018,13/09/2023 0:11:40,36227895,CF01148L100MA2,LIBERTAD 2,"-34,7097","-58,7097",PEREYRA 50,2023-09-13 00:11:40
1377,1133597670,Conexion Movil,2412913006,13/09/2023 0:12:30,16821831,CF01212L100MA3,Brigada Aérea,"-34,700357","-58,737708",Curapaligue 1069,2023-09-13 00:12:30
...,...,...,...,...,...,...,...,...,...,...,...
161688,1138094771,Conexion Movil,2790711004,17/08/2023 22:49:46,146412637,CF01148L200MB3,Libertad 2,"-34,712134","-58,710058",PEREYRA 50,2023-08-17 22:49:46
163753,1126809352,Conexión Móvil,27463487,17/08/2023 23:09:31,2442,CF05175U100MA1,Crucero Belgrano 3,"-34,703730","-58,724950",BEBEDERO 1992,2023-08-17 23:09:31
164501,1132543437,Conexion Movil,108808007,17/08/2023 23:17:49,749297736,CF01212L100MA3,Brigada Aérea,"-34,700357","-58,737708",Curapaligue 1069,2023-08-17 23:17:49
167587,1126903295,Conexion Movil,76098024,17/08/2023 23:49:10,48895474,CF02081L100MB2,CRUCERO BELGRANO 2,"-34,722076","-58,724047",RP100(Av.Patricios) entre Murillo y Azul 1003,2023-08-17 23:49:10
